In [51]:
!pip install pymysql

import pandas as pd
import pymysql

# CSV 파일 경로
product_list_path = 'data/product_list_final.csv'
product_price_path = 'data/product_price_final.csv'
product_review_path = 'data/product_review_final.csv'
product_link_path = 'data/product_link_final.csv'
review_analysis_results_path = 'data/review_analysis_results.csv'


In [52]:
product_list_df = pd.read_csv(product_list_path)
product_price_df = pd.read_csv(product_price_path)
product_review_df = pd.read_csv(product_review_path)
product_link_df = pd.read_csv(product_link_path)
product_procon_df = pd.read_csv(review_analysis_results_path)

print(product_list_df.columns)
print(product_price_df.columns)
print(product_review_df.columns)
print(product_link_df.columns)
print(product_procon_df.columns)

Index(['product_id', 'product_brand', 'product_name', 'current_price',
       'regular_price', 'discount_rate', 'major_category', 'minor_category',
       'score_review', 'num_review'],
      dtype='object')
Index(['product_id', 'product_brand', 'product_name', 'price_time',
       'price_at'],
      dtype='object')
Index(['product_id', 'star_score', 'product_review'], dtype='object')
Index(['product_id', 'product_link', 'img_url', 'product_danawa_url'], dtype='object')
Index(['product_id', '장점', '단점'], dtype='object')


In [53]:
product_procon_df.rename(columns={'장점': 'pro', '단점': 'con'}, inplace=True)
product_procon_df.columns

Index(['product_id', 'pro', 'con'], dtype='object')

In [56]:
category_counts = product_list_df['major_category'].value_counts()
print(category_counts.head(6))

#만두, 즉석국, 밀키트 (Top 3 카테고리로 하기로함)

major_category
만두     269
즉석국    243
밀키트    233
치킨     170
튀김     107
떡갈비    103
Name: count, dtype: int64


In [57]:
db_config = {
    'host': '127.0.0.1',# MySQL 서버 주소
    'user': 'root',    # 사용자명
    'password': 'ghkdwlqls1!',# 비밀번호
    'database': 'ProductDB',    # 사용할 데이터베이스 이름
    'port': 3306                # 포트 번호 (기본값: 3306)
}

In [58]:
conn = pymysql.connect(**db_config)
cursor = conn.cursor()

In [59]:
table_name = 'Products'

cursor.execute(
    """
    SELECT
        TABLE_NAME
    FROM
        information_schema.tables
    WHERE
        TABLE_SCHEMA = %s
        AND TABLE_NAME = %s;
    """,
    (db_config['database'], table_name)  # 튜플 형태로 전달
)

result = cursor.fetchone()
if result:
    print(f"테이블 '{table_name}'이(가) 존재합니다.")
else:
    print(f"테이블 '{table_name}'이(가) 존재하지 않습니다.")

테이블 'Products'이(가) 존재합니다.


In [60]:
# 1. Products 테이블 데이터 삽입

product_insert_query = """
INSERT INTO Products (
    product_id, product_brand, product_name, product_link, img_url, 
    current_price, score_review, review_num, regular_price, discount_rate,
    major_category, minor_category, pro, con
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

# 필요한 모든 데이터프레임 병합
merged_df = (product_list_df
    .rename(columns={"num_review": "review_num"})  # num_review를 review_num으로 변경
    .merge(
        product_link_df[['product_id', 'product_link', 'img_url']], 
        on='product_id', 
        how='left'
    )
    .merge(
        product_procon_df.rename(columns={'장점': 'pro', '단점': 'con'})[['product_id', 'pro', 'con']], 
        on='product_id', 
        how='left'
    )
)

# NaN 값을 None으로 변환하는 함수
def convert_nan_to_none(value):
    return None if pd.isna(value) else value

# DataFrame에서 튜플 리스트로 변환
product_data = [
    tuple(convert_nan_to_none(value) for value in [
        row.product_id,
        row.product_brand,
        row.product_name,
        row.product_link,
        row.img_url,
        row.current_price,
        row.score_review,
        row.review_num,
        row.regular_price,
        row.discount_rate,
        row.major_category,
        row.minor_category,
        row.pro,
        row.con
    ])
    for row in merged_df.itertuples(index=False)
]

# 데이터 삽입
try:
    cursor.executemany(product_insert_query, product_data)
    conn.commit()
    print("데이터 삽입 성공!")
except Exception as e:
    conn.rollback()
    print(f"데이터 삽입 실패: {e}")

데이터 삽입 성공!


In [61]:
#product_list_final.csv 확인 -> 총 상품 개수 : 1400개
print(len(product_list_df['product_id'].unique()))

#product_review_final.csv 확인 -> 총 상품 개수 : 900개
print(len(product_review_df['product_id'].unique()))


1400
900


In [64]:
# 2. Product_Price_Change 테이블 데이터 삽입
price_insert_query = """
INSERT INTO Product_Price_Change (
    product_id, price_time, price_at
) VALUES (%s, %s, %s);
"""
product_price_df['price_time'] = pd.to_datetime(product_price_df['price_time'].apply(lambda x: f"2024-{x}"))

price_data = [
    (row.product_id, row.price_time.strftime('%Y-%m-%d %H:%M:%S'), row.price_at)
    for row in product_price_df.itertuples(index=False)
]

try:
    cursor.executemany(price_insert_query, price_data)
    conn.commit()
    print("데이터 삽입 성공!")
except Exception as e:
    conn.rollback()
    print(f"데이터 삽입 실패: {e}")

데이터 삽입 성공!


In [65]:
# 3. Product_Review 테이블 데이터 삽입
review_insert_query = """
INSERT INTO Product_Review (
    product_id, star_score, product_review
) VALUES (%s, %s, %s);
"""

# Rename column product_rate to star_score (필요한 경우)
product_review_df = product_review_df.rename(columns={"product_rate": "star_score"})

# DataFrame에서 튜플 리스트로 변환
review_data = [
    (row.product_id, row.star_score, row.product_review)
    for row in product_review_df.itertuples(index=False)
]

try:
    cursor.executemany(review_insert_query, review_data)
    conn.commit()
    print("데이터 삽입 성공!")
except Exception as e:
    conn.rollback()
    print(f"데이터 삽입 실패: {e}")

데이터 삽입 성공!
